In [ ]:
import numpy as np
import pandas as pd
import shap
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

In [ ]:
import tensorflow as tf
print(tf.__version__)

In [ ]:
# Load the data
df = pd.read_csv("sampled_10_percent.csv", delimiter=',')
df['public_date'] = pd.to_datetime(df['public_date'])
df = df.sort_values(by=['permno', 'public_date'])

In [ ]:
# Get unique company counts before filtering
initial_bankrupt_count = df[df['Bankrupt']].drop_duplicates('permno').shape[0]
initial_non_bankrupt_count = df[~df['Bankrupt']].drop_duplicates('permno').shape[0]

# Specify the periods, adjusted for zero-based index
periods = [x - 1 for x in range(3, 37, 3)]  # Simplified generation of periods

# Filter rows for each 'permno' based on the specified periods
filtered_df = df.groupby('permno').nth(periods).reset_index()

# Count unique bankrupt and non-bankrupt companies post-filtering
filtered_bankrupt_count = filtered_df[filtered_df['Bankrupt']].drop_duplicates('permno').shape[0]
filtered_non_bankrupt_count = filtered_df[~filtered_df['Bankrupt']].drop_duplicates('permno').shape[0]

# Print the initial and post-filtering counts
print(f"Initial Number of bankrupt companies: {initial_bankrupt_count}")
print(f"Initial Number of non-bankrupt companies: {initial_non_bankrupt_count}")
print(f"Filtered Number of bankrupt companies: {filtered_bankrupt_count}")
print(f"Filtered Number of non-bankrupt companies: {filtered_non_bankrupt_count}")

In [ ]:
# Ratios based on Feature Selection using LASSO
ratios = ['X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'X9', 'X10',
          'X11', 'X12', 'X13', 'X14', 'X15', 'X16', 'X17', 'X18', 'X19',
          'X20', 'X21', 'X22', 'X23', 'X24', 'X25', 'X26', 'X27', 'X28',
          'X29', 'X30', 'X31', 'X32', 'X33', 'X34', 'X35', 'X36', 'X37',
          'X38', 'X39', 'X40', 'X41', 'X42', 'X43', 'X44', 'X45', 'X46',
          'X47', 'X48', 'X49', 'X50', 'X51', 'X52', 'X53', 'X54', 'X55',
          'X56', 'X57', 'X58', 'X59', 'X60', 'X61', 'X62', 'X63', 'X64',
          'X65', 'X66', 'X67', 'X68', 'X69', 'X70', 'X71']

In [ ]:
# Prepare the dataset for LSTM
X, y, permnos = [], [], []
grouped = df.groupby('permno')

sequence_length = 12
for permno, group in grouped:
    group = group.sort_values(by='public_date')  # Ensure sorting by date
    if len(group) >= sequence_length:
        X.append(group[ratios].tail(sequence_length).values)
        y.append(group['Bankrupt'].iloc[-1])
        permnos.append(permno)  # Store permno for later use

X, y = np.array(X), np.array(y)

In [ ]:
# Impute missing values
imputer = SimpleImputer(strategy='mean')
X = imputer.fit_transform(X.reshape(-1, X.shape[-1])).reshape(X.shape)


In [ ]:
# Normalize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X.reshape(-1, X.shape[-1])).reshape(X.shape)


In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
# Load the model
model = load_model('LSTM_Model_72_Q36.h5')

In [ ]:
# Prepare a subset of the training data 
background_data = X_scaled[:1000]

# Initialize the SHAP DeepExplainer with the model and the background data
explainer = shap.DeepExplainer(model, background_data)

# Prepare a subset of test data for which you want explanations
test_data_for_explanation = X_scaled

In [ ]:
# Calculate SHAP values for the selected test data
shap_values = explainer.shap_values(test_data_for_explanation)

# Assuming the model outputs SHAP values for each class and is a classification model
selected_class_index = 0  # Assuming the bankrupt class is at index 1
shap_values_for_class = shap_values[selected_class_index]


In [ ]:
# For LSTM or models where input is sequence data, we want to focus on the last timestep for SHAP values
# This focuses on the last timestep's SHAP values if the model is an LSTM
if len(shap_values_for_class.shape) == 3:
    shap_values_last_timestep = shap_values_for_class[:, -1, :]
    test_data_last_timestep = test_data_for_explanation[:, -1, :]
else:
    shap_values_last_timestep = shap_values_for_class
    test_data_last_timestep = test_data_for_explanation

# Print shapes to confirm the adjustment
print("Adjusted SHAP values shape:", shap_values_last_timestep.shape)
print("Adjusted test data shape:", test_data_last_timestep.shape)

# Try plotting again with matched number of rows
try:
    shap.summary_plot(shap_values_last_timestep, test_data_last_timestep, feature_names=ratios, plot_type='bar')
    shap.summary_plot(shap_values_last_timestep, test_data_last_timestep, feature_names=ratios)
    print("Successfully generated SHAP summary plot for bankrupt class.")
except Exception as e:
    print("Error in generating SHAP summary plot for bankrupt class:", e)


In [ ]:
# For LSTM or models where input is sequence data, we want to focus on the last timestep for SHAP values
if len(shap_values_for_class.shape) == 3:
    shap_values_last_timestep = shap_values_for_class[:, -1, :]
    test_data_last_timestep = test_data_for_explanation[:, -1, :]
else:
    shap_values_last_timestep = shap_values_for_class
    test_data_last_timestep = test_data_for_explanation

# Function to remove outliers using quantiles
def remove_outliers(values, low_quantile=0.01, high_quantile=0.99):
    lower_bound = np.quantile(values, low_quantile)
    upper_bound = np.quantile(values, high_quantile)
    mask = (values >= lower_bound) & (values <= upper_bound)
    return mask

# Select a feature for the dependence plot
feature_to_plot = 'X13'
feature_index = ratios.index(feature_to_plot)

# Remove outliers based on the `bm` feature itself
bm_values = test_data_last_timestep[:, feature_index]
mask_bm = remove_outliers(bm_values, 0.05, 0.95)
filtered_bm_values = bm_values[mask_bm]
filtered_test_data = test_data_last_timestep[mask_bm]
filtered_shap_values = shap_values_last_timestep[mask_bm]

# Further filter outliers from SHAP values themselves
mask_shap = remove_outliers(filtered_shap_values[:, feature_index], 0.01, 0.99)
final_filtered_shap_values = filtered_shap_values[mask_shap]
final_filtered_test_data = filtered_test_data[mask_shap]

# Create the SHAP dependence plot
try:
    shap.dependence_plot(
        feature_to_plot,                           # Feature to plot
        final_filtered_shap_values,                # SHAP values for the selected class (outliers removed)
        final_filtered_test_data,                  # Corresponding features from the test data (outliers removed)
        feature_names=ratios,                      # Names of the features
        interaction_index=None                     # Set to None for no interaction, or specify another feature name
    )
    print(f"Successfully generated SHAP dependence plot for {feature_to_plot}.")
except Exception as e:
    print(f"Error in generating SHAP dependence plot for {feature_to_plot}:", e)

In [ ]:
# For LSTM models, extract the SHAP values for the last timestep
shap_values_for_class = np.array([sv[-1] for sv in shap_values[0]])

# Print shapes to confirm the adjustment
print("Adjusted SHAP values shape:", shap_values_for_class.shape)


In [ ]:
desired_permno = 80803  # Set the desired permno here

# Function to format the bar values with more precision
def format_bar_labels(ax, precision=4):
    for text in ax.texts:
        try:
            # Check if the text is a number and format it
            value = float(text.get_text())
            text.set_text(f'{value:.{precision}f}')
        except ValueError:
            continue

# Find the index of the desired permno in the test_permnos list
if desired_permno in permnos:
    instance_index = permnos.index(desired_permno)
    try:
        shap.initjs()  # Initialize JavaScript visualization in Jupyter notebook

        # Convert the SHAP values to an Explanation object
        feature_data = pd.DataFrame(test_data_for_explanation[instance_index], columns=ratios)
        explanation = shap.Explanation(values=shap_values_for_class[instance_index],
                                       base_values=explainer.expected_value[0],  # Assuming single output model
                                       data=feature_data.iloc[0],
                                       feature_names=ratios)

        # Plotting the waterfall chart
        fig, ax = plt.subplots()
        shap.plots.waterfall(explanation, show=False)

        # Get current y-tick labels
        yticklabels = ax.get_yticklabels()
        # Extract feature names without the numbers
        new_labels = [label.get_text().split('=')[1].strip() if '=' in label.get_text() else label.get_text() for label in yticklabels]
        
        # Apply new labels
        ax.set_yticklabels(new_labels)

        # Format the bar labels to show more precision
        format_bar_labels(ax, precision=6)

        plt.title(f'SHAP Waterfall Plot for permno: {desired_permno}')  # Display permno in the plot title
        plt.show()
        print("Successfully generated SHAP waterfall plot for permno:", desired_permno)
    except Exception as e:
        print("Error in generating SHAP waterfall plot for permno:", desired_permno, ":", e)
else:
    print("Permno", desired_permno, "not found in the test data.")

In [ ]:
desired_permno = 80803  # Set the desired permno here

# Find the index of the desired permno in the test_permnos list
if desired_permno in permnos:
    instance_index = permnos.index(desired_permno)
    if instance_index is not None:
        try:
            shap.initjs()  # Initialize JavaScript visualization in Jupyter notebook

            # Data for the instance
            feature_data = test_data_for_explanation[instance_index]

            # Visualize the force plot for the specific instance
            plot = shap.force_plot(
                explainer.expected_value[0],  # Base value from the explainer
                shap_values[0][instance_index],  # SHAP values for the desired instance and class
                feature_data,  # Feature data for the desired instance
                feature_names=ratios  # Names of the features
            )
            
            # Display the plot directly in the notebook
            shap.save_html(f"SHAP_force_plot_permno_{desired_permno}.html", plot)  # Save the plot as an HTML file
            display(plot)
            print(f"Successfully generated SHAP force plot for permno: {desired_permno}")
        except Exception as e:
            print(f"Error in generating SHAP force plot for permno: {desired_permno}: {e}")
else:
    print(f"Permno {desired_permno} not found in the test data.")
